In [43]:
import fastparse._

import fastparse._

The grammar of µDhall is a much simplified version of the [Dhall grammar](https://github.com/dhall-lang/dhall-lang/blob/master/standard/dhall.abnf).

In [ ]:
object Grammar {
    
}